In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


In [2]:
# Bernstein–Vazirani Algorithm using Qiskit 2.x
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle(qc, inputs, ancilla, s):
    """Implements oracle for f(x) = s · x (no constant b)."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    fig = plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most

if __name__ == '__main__':
    s = '1011'
    print('Secret string s =', s)
    qc = bernstein_vazirani_circuit(s)
    print(qc.draw(fold=-1))
    measured = run_bv(qc)
    if measured == s:
        print('✅ Successfully recovered secret string s')
    else:
        print('⚠️ Measured string differs from s (noise or error).')


Secret string s = 1011
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101
⚠️ Measured string differs from s (noise or error).


### Task 1

In [3]:
# Change the secret string s and run the circuit again
s_new = '0110'
print('New secret string s =', s_new)
qc_new = bernstein_vazirani_circuit(s_new)
print(qc_new.draw(fold=-1))
measured_new = run_bv(qc_new)
if measured_new == s_new:
    print('✅ Successfully recovered new secret string s')
else:
    print('⚠️ Measured string differs from new s (noise or error).')

New secret string s = 0110
     ┌───┐┌───┐     ┌─┐                     
q_0: ┤ H ├┤ H ├─────┤M├─────────────────────
     ├───┤└───┘     └╥┘┌───┐          ┌─┐   
q_1: ┤ H ├───────■───╫─┤ H ├──────────┤M├───
     ├───┤       │   ║ └───┘     ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼───╫────────■──┤ H ├─╫─┤M├
     ├───┤┌───┐  │   ║  ┌─┐   │  └───┘ ║ └╥┘
q_3: ┤ H ├┤ H ├──┼───╫──┤M├───┼────────╫──╫─
     ├───┤├───┤┌─┴─┐ ║  └╥┘ ┌─┴─┐      ║  ║ 
q_4: ┤ X ├┤ H ├┤ X ├─╫───╫──┤ X ├──────╫──╫─
     └───┘└───┘└───┘ ║   ║  └───┘      ║  ║ 
c: 4/════════════════╩═══╩═════════════╩══╩═
                     0   3             1  2 
Counts: {'0110': 1024}
Most frequent measured bitstring (input register): 0110
✅ Successfully recovered new secret string s


### Task 2

In [4]:
# Modified Bernstein–Vazirani Algorithm with constant bit b
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle_with_b(qc, inputs, ancilla, s, b):
    """Implements oracle for f(x) = s · x ⊕ b."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    if b == '1':
        qc.x(ancilla)

def bernstein_vazirani_circuit_with_b(s, b):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle_with_b(qc, inputs, ancilla, s, b)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_bv_with_b(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    fig = plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most

In [5]:
# Run the modified circuit with b=0
s = '1011'
b = '0'
print(f'Secret string s = {s}, Constant bit b = {b}')
qc_with_b_0 = bernstein_vazirani_circuit_with_b(s, b)
print(qc_with_b_0.draw(fold=-1))
measured_with_b_0 = run_bv_with_b(qc_with_b_0)
if measured_with_b_0 == s:
    print('✅ Successfully recovered secret string s (with b=0)')
else:
    print('⚠️ Measured string differs from s (noise or error).')

Secret string s = 1011, Constant bit b = 0
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101
⚠️ Measured string differs from s (noise or error).


In [6]:
# Run the modified circuit with b=1
s = '1011'
b = '1'
print(f'Secret string s = {s}, Constant bit b = {b}')
qc_with_b_1 = bernstein_vazirani_circuit_with_b(s, b)
print(qc_with_b_1.draw(fold=-1))
measured_with_b_1 = run_bv_with_b(qc_with_b_1)
if measured_with_b_1 == s:
    print('✅ Successfully recovered secret string s (with b=1)')
else:
    print('⚠️ Measured string differs from s (noise or error).')

Secret string s = 1011, Constant bit b = 1
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ ├───┤ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫─┤ X ├─╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║ └───┘ ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101
⚠️ Measured string differs from s (noise or error).


### Task 4

In [7]:
# Import necessary libraries for noise simulation
from qiskit_aer.noise import NoiseModel, depolarizing_error, pauli_error
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

In [8]:
# Create a simple noise model
# Add a depolarizing error to all single qubit gates
error_1q = depolarizing_error(0.01, 1)
# Add a depolarizing error to all two qubit gates
error_2q = depolarizing_error(0.05, 2)

# Create a noise model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1q, ['h', 'x'])
noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])

print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'h', 'id', 'rz', 'sx', 'x']
  Instructions with noise: ['x', 'h', 'cx']
  All-qubits errors: ['h', 'x', 'cx']


In [9]:
# Run the modified circuit with noise
s = '1011'
b = '0' # You can change b to '1' to see its effect with noise
print(f'Secret string s = {s}, Constant bit b = {b}')
qc_noisy = bernstein_vazirani_circuit_with_b(s, b)
print(qc_noisy.draw(fold=-1))
measured_noisy = run_bv_with_b(qc_noisy, noise_model=noise_model)
if measured_noisy == s:
    print('✅ Successfully recovered secret string s (with noise)')
else:
    print('⚠️ Measured string differs from s (noise or error).')

Secret string s = 1011, Constant bit b = 0
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 


TypeError: run_bv_with_b() got an unexpected keyword argument 'noise_model'